
# Advanced Certification in AIML
## A Program by IIIT-H and TalentSprint


## Learning Objective

At the end of the experiment, you will be able to :

* perform sentiment classification from movie reviews, using a Recurrent Neural Network (RNN)



In [ ]:
#@title Experiment Walkthrough Video

from IPython.display import HTML

HTML("""<video width="850" height="480" controls>
  <source src="https://cdn.talentsprint.com/talentsprint1/archives/sc/aiml/movie_sentiment_analysis.mp4" type="video/mp4">
</video>
""")

## Recurrent neural network (RNN)

RNN is a type of neural network that is proven to work well with sequence data. Since text is actually a sequence of words, a recurrent neural network is an automatic choice to solve text-related problems. Here, we will use an LSTM (Long Short Term Memory network) which is a variant of RNN, to solve a movie reviews based sentiment classification problem. 

An LSTM unit consists of a cell, an input gate, an output gate and a forget gate. The cell remembers values over arbitrary time intervals and the three gates regulate the flow of information into and out of the cell.

LSTM networks are well-suited to classification based on time series data and deal well with the exploding and vanishing gradient problems that can be encountered when training traditional RNNs

<img style="-webkit-user-select: none;margin: auto;" src="https://miro.medium.com/max/1302/1*yr0820z7YNRcDCWGisLC4g.png" width="450" height="250">


<img style="-webkit-user-select: none;margin: auto;" src="https://miro.medium.com/max/1276/1*mvxPFvnDqj2jJrsjevD41A.png" width="450" height="250">







## Dataset Description

The IMDB dataset from [stanford](http://ai.stanford.edu/~amaas/data/sentiment/) has around 50K movie reviews with the following attributes:
* **review:** review of any movie
* **sentiment:** positive or negative sentiment value

The dataset contains equal number of positive and negative sentiment reviews


In [ ]:
! wget -qq https://cdn.talentsprint.com/aiml/Experiment_related_data/IMDB_Dataset.csv


### Importing required packages

In [ ]:
import pandas as pd
import numpy as np
import re
from numpy import array
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer

### Dataset analysis

In [ ]:
movie_reviews = pd.read_csv("IMDB_Dataset.csv")

movie_reviews.isnull().values.any()

In [ ]:
print(movie_reviews.shape)
movie_reviews.head()

In [ ]:
# Let us view one of the reviews
movie_reviews["review"][5]

### Data Preprocessing

Remove html tags, punctuations, special characters etc. from the review text

In [ ]:
# Data Preprocessing 
def preprocess_text(sen):
    # PRE-PROCESS A GIVEN REVIEW
    sen = re.sub('<.*?>', ' ', sen) # remove html tag
    sen = re.sub('[^a-zA-Z]', ' ', sen)    # remove non alphabet
    sen = re.sub(r"\s+[a-zA-Z]\s+", ' ', sen)    # remove Single characters
    sen = re.sub(r'\s+', ' ', sen)     # remove multiple spaces
    sen = sen.lower() # lower case
    return sen

In [ ]:
# Store the preprocessed reviews in a new list
X = []
sentences = list(movie_reviews['review'])
for sen in sentences:
    X.append(preprocess_text(sen))

In [ ]:
print(X[5])

In [ ]:
# Convert labels to integers
y = movie_reviews['sentiment']
y = np.array(list(map(lambda x: 1 if x=="positive" else 0, y)))

### Training and Testing

In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

print('Train Set',len(X_train))
print('Test Set',len(X_test))

In [ ]:
print(X_train[0])

### Prepare the Embedding Layer

The embedding layer converts our textual data into a dense vector representation and is used as the first layer for the deep learning models in Keras

Steps:

1. Tokenization 

2. Padding

3. Embedding

In [ ]:
# Tokenizer class from the keras.preprocessing.text module creates a word-to-index integer dictionary
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [ ]:
print(np.asarray(X_train).shape)
print(np.asarray(X_test).shape)

The X_train set contains 40,000 lists of integers, each list corresponding to the sentences in a review. Set the maximum length of each list to 100 and add 0 padding to those lists that have a length < 100, until they reach a length of 100

In [ ]:
# Perform padding on both train and test set
max_length = 100

X_train = pad_sequences(X_train, maxlen=max_length, padding='post', truncating='post')
X_test = pad_sequences(X_test, maxlen=max_length, padding='post', truncating='post')

In [ ]:
print('Encoded X Train\n', X_train, '\n')
print('Encoded X Test\n', X_test, '\n')
print('Maximum review length: ', max_length)

### Build LSTM Model for text classification

In [ ]:
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Embedding, LSTM
from tensorflow.python.keras.layers.embeddings import Embedding

EMBEDDING_DIM = 100

print('Build model...')

model = Sequential()
model.add(Embedding(5000, EMBEDDING_DIM, input_length = max_length))
model.add(LSTM(units=32,  dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

print('Summary of the built model...')
print(model.summary())

### Training and Validation

In [ ]:
history = model.fit(X_train, y_train, batch_size=128, epochs=5, verbose=2, validation_split=0.2)

### Testing

In [ ]:
print('Testing...')
y_test = np.array(y_test)
loss, accuracy = model.evaluate(X_test, y_test, batch_size=128)

print('Test loss:', loss)
print('Test accuracy:', accuracy)
print("Accuracy: {0:.2%}".format(accuracy))

In [ ]:
# Let us test some  samples
test_sample_1 = "This movie is fantastic! I really like it because it is so good!"
test_sample_2 = "Good movie!"
test_sample_3 = "Maybe I like this movie."
test_sample_4 = "Not to my taste, will skip and watch another movie"
test_sample_5 = "if you like action, then this movie might be good for you."
test_sample_6 = "Bad movie!"
test_sample_7 = "Not a good movie!"
test_sample_8 = "This movie really sucks! Can I get my money back please?"
test_samples = [test_sample_1, test_sample_2, test_sample_3, test_sample_4, test_sample_5, test_sample_6, test_sample_7, test_sample_8]

for each in test_samples:
  filtered = [preprocess_text(each)]
  tokenize_words = tokenizer.texts_to_sequences(filtered)
  tokenize_words = pad_sequences(tokenize_words, maxlen=max_length, padding='post', truncating='post')
  result = model.predict(tokenize_words)
  if result >= 0.7:
      print('positive == ',each)
  else:
      print('negative == ',each)